In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

from aind_vr_foraging_analysis.utils import parse
from aind_vr_foraging_analysis.utils import breathing_signal as breathing

# Plotting libraries
import matplotlib.pyplot as plt
import pytz

import seaborn as sns
import pandas as pd
from datetime import datetime

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'
foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents'



In [ ]:
def parse_user_date(user_date_str):
    """
    Parses a user-provided date string in the format 'YYYY-MM-DD' and returns a datetime.date object.

    Parameters:
    user_date_str (str): A string representing a date in the format 'YYYY-MM-DD'.

    Returns:
    datetime.date: The parsed date if the format is valid.
    None: If the input format is incorrect.
    """
    try:
        return datetime.strptime(user_date_str, "%Y-%m-%d").date()  # Convert user input to date
    except ValueError:
        return None

def extract_and_convert_time(filename):
    """
    Extracts a timestamp from a filename and converts it to a local date in the 'America/Los_Angeles' timezone.

    The filename must follow one of these formats:
    - 'prefix_YYYY-MM-DDTHHMMSSZ_suffix' (UTC timestamp, indicated by 'Z')
    - 'prefix_YYYYMMDDTHHMMSS_suffix' (Local time in 'America/Los_Angeles')

    Parameters:
    filename (str): A string containing a timestamp in one of the expected formats.

    Returns:
    datetime.date: The extracted and converted local date.
    str: "Invalid filename format" if the filename format does not match expectations.
    """
    seattle_tz = pytz.timezone('America/Los_Angeles')

    # Extract the timestamp part
    timestamp_part = filename.split("_")[1]

    try:
        if "Z" in timestamp_part:  # Case: UTC timestamp
            dt_utc = datetime.strptime(timestamp_part, "%Y-%m-%dT%H%M%SZ")
            dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(seattle_tz)
        else:  # Case: Already local time
            dt_local = datetime.strptime(timestamp_part, "%Y%m%dT%H%M%S")
            dt_local = seattle_tz.localize(dt_local)
        return dt_local.date()
    except ValueError:
        return "Invalid filename format"


In [ ]:
# Day with the old and working cable
date_string = "2025-3-21"
date = parse_user_date(date_string)
mouse = '789924'

In [ ]:
session_found = False
session_n = 0
directory = os.path.join(base_path, mouse)
files = os.listdir(os.path.join(base_path, mouse))

sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)

# All this segment is to find the correct session without having the specific path
for file_name in sorted_files:
    
    if session_found == True:
        break
    
    session = extract_and_convert_time(file_name)
    print(session)
    if session != date:
        continue
    else:
        print(str(session), file_name)
        session_found = True
        
    # Recover data streams
    session_path = os.path.join(base_path, mouse, file_name)
    session_path = Path(session_path)
    data = parse.load_session_data(session_path)
    # Load the encoder data separately
    stream_data = parse.ContinuousData(data)

In [ ]:
signal = stream_data.breathing.data.values
zero_index = stream_data.breathing.index[0]

# Define filter parameters
cutoff_freq = 0.5  # cutoff frequency in Hz
fs = 250  # sampling frequency in Hz

# Apply high-pass filter
filtered_breathing = breathing.highpass_filter(signal, cutoff_freq, fs)

cutoff_freq = 40  # cutoff frequency in Hz
filtered_breathing = breathing.lowpass_filter(filtered_breathing, cutoff_freq, fs)

## Signals with bandpass filter (1-40Hz)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(15,8), sharey=True)
plt.suptitle(f'{mouse}')

i=0
for ax in axs.flatten():
    ax.plot(stream_data.breathing.index-zero_index, filtered_breathing, color= 'black')

    ax.set_ylim(-3000, 3000)
    ax.set_xlim(i, 5+i)
    ax.set_ylabel('Breathing (a.u.)')
    ax.set_xlabel('Time (s)')
    i+=10

sns.despine()
plt.tight_layout()
fig.savefig(os.path.join(foraging_figures, f'{mouse}_thermistor_screening.png'), bbox_inches='tight')

## Using a running average and subtracting the signals

In [ ]:
window_size=30
slow_ther = breathing.moving_average(filtered_breathing, window_size=window_size)
new_ther = filtered_breathing-slow_ther

fig, axs = plt.subplots(2,2, figsize=(15,8), sharey=True)
plt.suptitle(f'{mouse}')

i=0
for ax in axs.flatten():
    ax.plot(stream_data.breathing.index-zero_index, new_ther, color= 'black')
    # ax.plot(stream_data.breathing.index-zero_index, slow_ther, color= 'blue')
    # ax.set_ylim(-100, 100)
    ax.set_xlim(0+i, 5+i)
    ax.set_ylabel('Breathing (a.u.)')
    ax.set_xlabel('Time (s)')
    i+=10

sns.despine()
plt.tight_layout()
fig.savefig(os.path.join(foraging_figures, f'{mouse}_thermistor_screening_with_moving_average.png'), bbox_inches='tight')
